In [11]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

from flavors.activate_function import ActivateFunctionController, ActivateFunctions

In [12]:
Lx = 5.0
Ly = 1.0
T = 1.0
nu = 0.01

U = 1



class NavierStokesModel(nn.Module):
    def __init__(self):
        super(NavierStokesModel, self).__init__()

        self.activation_func = ActivateFunctionController(
            activate_func=ActivateFunctions.AdaptiveBlendingUnit, args=dict()
        ).get()

        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 3)

    def forward(self, x):
        x = self.activation_func(self.fc1(x))
        x = self.activation_func(self.fc2(x))
        x = self.activation_func(self.fc3(x))
        return self.fc4(x)

global_epoch = 0
writer = SummaryWriter()
model = NavierStokesModel()


def compute_pde(xyt):
    global global_epoch
    xyt.requires_grad_(True)
    up = model(xyt)
    u, v, p = up[:, 0], up[:, 1], up[:, 2]

    u_x = torch.autograd.grad(u.sum(), xyt, create_graph=True)[0][:, 0]
    u_y = torch.autograd.grad(u.sum(), xyt, create_graph=True)[0][:, 1]
    u_t = torch.autograd.grad(u.sum(), xyt, create_graph=True)[0][:, 2]
    v_x = torch.autograd.grad(v.sum(), xyt, create_graph=True)[0][:, 0]
    v_y = torch.autograd.grad(v.sum(), xyt, create_graph=True)[0][:, 1]
    v_t = torch.autograd.grad(v.sum(), xyt, create_graph=True)[0][:, 2]
    p_x = torch.autograd.grad(p.sum(), xyt, create_graph=True)[0][:, 0]
    p_y = torch.autograd.grad(p.sum(), xyt, create_graph=True)[0][:, 1]

    u_xx = torch.autograd.grad(u_x.sum(), xyt, create_graph=True)[0][:, 0]
    u_yy = torch.autograd.grad(u_y.sum(), xyt, create_graph=True)[0][:, 1]
    v_xx = torch.autograd.grad(v_x.sum(), xyt, create_graph=True)[0][:, 0]
    v_yy = torch.autograd.grad(v_y.sum(), xyt, create_graph=True)[0][:, 1]

    continuity = u_x + v_y

    momentum_x = u_t + u * u_x + v * u_y + p_x - nu * (u_xx + u_yy)
    momentum_y = v_t + u * v_x + v * v_y + p_y - nu * (v_xx + v_yy)


    writer.add_scalar("pde/continuity", torch.abs(torch.mean(continuity)), global_epoch)
    writer.add_scalar("pde/momentum_x", torch.abs(torch.mean(momentum_x)), global_epoch)
    writer.add_scalar("pde/momentum_y", torch.abs(torch.mean(momentum_y)), global_epoch)

    return continuity, momentum_x, momentum_y


def boundary_conditions():
    num_points = 1000
    t = T * np.random.random(num_points)
    bottom_bc = torch.tensor(
        np.stack(
            [np.random.uniform(0, Lx, num_points), np.zeros(num_points), t], axis=-1
        ),
        requires_grad=False,
    ).float()
    left_bc = torch.tensor(
        np.stack(
            [np.zeros(num_points), np.random.uniform(0, Ly, num_points), t], axis=-1
        ),
        requires_grad=False,
    ).float()
    top_bc = torch.tensor(
        np.stack(
            [np.random.uniform(0, Lx, num_points), np.full(num_points, Ly), t], axis=-1
        ),
        requires_grad=False,
    ).float()
    right_bc = torch.tensor(
        np.stack(
            [np.full(num_points, Lx), np.random.uniform(0, Ly, num_points), t], axis=-1
        ),
        requires_grad=False,
    ).float()

    bottom_predict = model(bottom_bc)
    left_predict = model(left_bc)
    top_predict = model(top_bc)
    right_predict = model(right_bc)

    u, v, p = left_predict[:, 0], left_predict[:, 1], left_predict[:, 2]
    left_loss = torch.mean(p**2)
    u, v, p = top_predict[:, 0], top_predict[:, 1], top_predict[:, 2]
    top_loss = torch.mean((u - U)**2 + v**2)
    u, v, p = bottom_predict[:, 0], bottom_predict[:, 1], bottom_predict[:, 2]
    bottom_loss = torch.mean(u**2 + v**2)
    u, v, p = right_predict[:, 0], right_predict[:, 1], right_predict[:, 2]
    right_loss = torch.mean(p**2)

    writer.add_scalar("bc/bottom", torch.mean(bottom_loss), global_epoch)
    writer.add_scalar("bc/left", torch.mean(left_loss), global_epoch)
    writer.add_scalar("bc/right", torch.mean(right_loss), global_epoch)
    writer.add_scalar("bc/top", torch.mean(top_loss), global_epoch)

    return right_loss + bottom_loss + top_loss +left_loss


def generate_data(num_points):
    x = np.random.uniform(0, Lx, num_points)
    y = np.random.uniform(0, Ly, num_points)
    t = np.random.uniform(0, T, num_points)
    xyt = np.stack([x, y, t], axis=-1)
    return torch.tensor(xyt, requires_grad=True).float()


def loss_function(xyt):
    continuity, momentum_x, momentum_y = compute_pde(xyt)
    pde_loss = (
        torch.mean(continuity**2)
        + torch.mean(momentum_x**2)
        + torch.mean(momentum_y**2)
    )
    bc_loss = boundary_conditions()
    total_loss = pde_loss + bc_loss
    return total_loss


In [13]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


num_epochs = 8000
num_points = 500
for epoch in range(num_epochs):
    global_epoch += 1
    xyt = generate_data(num_points)
    optimizer.zero_grad()
    loss = loss_function(xyt)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training completed.")


Epoch 0, Loss: 0.8351815342903137
Epoch 100, Loss: 0.002486208453774452
Epoch 200, Loss: 0.0007067625410854816
Epoch 300, Loss: 0.0003842358128167689
Epoch 400, Loss: 0.00026178581174463034
Epoch 500, Loss: 0.00017383205704391003
Epoch 600, Loss: 0.00013504951493814588
Epoch 700, Loss: 0.00010665427544154227
Epoch 800, Loss: 8.65926340338774e-05
Epoch 900, Loss: 7.070029823808e-05
Epoch 1000, Loss: 5.734114529332146e-05
Epoch 1100, Loss: 4.844922659685835e-05
Epoch 1200, Loss: 3.54194053215906e-05
Epoch 1300, Loss: 3.278957592556253e-05
Epoch 1400, Loss: 2.9494805858121254e-05
Epoch 1500, Loss: 2.367371052969247e-05
Epoch 1600, Loss: 2.0498016965575516e-05
Epoch 1700, Loss: 1.7124937585322186e-05
Epoch 1800, Loss: 3.87069521821104e-05
Epoch 1900, Loss: 1.5494399121962488e-05
Epoch 2000, Loss: 1.1533819815667812e-05
Epoch 2100, Loss: 1.0844717508007307e-05
Epoch 2200, Loss: 1.1725364856829401e-05
Epoch 2300, Loss: 1.8627280951477587e-05
Epoch 2400, Loss: 8.728684406378306e-06
Epoch 2500

In [14]:
def plot_flow_data(model, writer: SummaryWriter):
    nx, ny, nt = 50, 50, 4
    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)
    t = np.linspace(0, T, nt)
    X, Y, t_grid = np.meshgrid(x, y, t)
    XYT = np.stack([X.flatten(), Y.flatten(), t_grid.flatten()], axis=-1)
    XYT_tensor = torch.tensor(XYT, requires_grad=False).float()

    X, Y = np.meshgrid(x, y)
    with torch.no_grad():
        predictions = model(XYT_tensor).cpu().numpy()

    U_calc = predictions[:, 0].reshape((ny, nx, nt))
    V_calc = predictions[:, 1].reshape((ny, nx, nt))
    # P_calc = predictions[:, 2].reshape((ny, nx, nt))

    U_exact = np.tile(U / Ly * y, (nx)).reshape(nx, ny).T
    V_exact = 0 * y[None, :] * x[:, None]

    U_error = np.abs(U_exact - U_calc[:, :, 0])
    V_error = np.abs(V_exact - V_calc[:, :, 0])

    error = np.sqrt(U_error**2 + V_error**2)

    fig = plt.figure(figsize=(15, 5))
    ax = fig.add_subplot(111)
    c3 = ax.contourf(X, Y, error, levels=50, cmap="viridis")
    ax.set_title("Error")
    fig.colorbar(c3)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

    plt.tight_layout()
    writer.add_figure("total/error", figure=fig, global_step=global_epoch)

    plt.close()


In [15]:
plot_flow_data(model, writer)
